In [1]:
import pandas as pd
import numpy as np
import cirpy


In [2]:
compound_fp = './database/Compound.csv'


content_fp = './database/Content.csv'
drug_info_comb_fp = './data/Drug_info_combined.csv'
food_fp = './database/Food.csv'

pd.set_option('display.max_columns', None)

In [3]:
# df = pd.read_csv(food_fp, chunksize = 16)
# for i in df:
#     print(i.iloc[0, :])
#     break

In [4]:
content_df = pd.read_csv(content_fp, chunksize = 16)
for i in content_df:
    display(i.head(2))
    break

,id,source_id,source_type,food_id,orig_food_id,orig_food_common_name,orig_food_scientific_name,orig_food_part,orig_source_id,orig_source_name,orig_content,orig_min,orig_max,orig_unit,orig_citation,citation,citation_type,creator_id,updater_id,created_at,updated_at,orig_method,orig_unit_expression,standard_content,preparation_type,export
0,1,1,Nutrient,4,29,Kiwi,Actinidia chinensis PLANCHON [Actinidiaceae],Fruit,FAT,FAT,1955.00,70.0,3840.0,mg/100g,NaN,DUKE,DATABASE,NaN,NaN,2014-11-05 13:42:11 UTC,2019-12-11 20:23:57 UTC,NaN,NaN,1955.00,raw,0
1,2,1,Nutrient,6,53,Onion,Allium cepa L. [Liliaceae],Bulb,FAT,FAT,1853.95,100.0,3607.9,mg/100g,NaN,DUKE,DATABASE,NaN,NaN,2014-11-05 13:42:11 UTC,2019-12-11 20:23:57 UTC,NaN,NaN,1853.95,raw,0


In [5]:
compound_df = pd.read_csv(compound_fp)
# for i in compound_df:
#     display(i.head(2).T)
#     tmp = i.loc[i.description == '350602-26-5', 'cas_number'].values
#     display(tmp)
#     break

/Applications/anaconda3/envs/deepddi2/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (3,4,5,6,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
drug_info_comb = pd.read_csv(drug_info_comb_fp, index_col = 'Unnamed: 0')
drug_info_comb

,Drug name,Name,Smiles
0,DB00006,Bivalirudin,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...
1,DB00014,Goserelin,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...
2,DB00035,Desmopressin,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...
3,DB00050,Cetrorelix,CC(C)C[C@H](NC(=O)[C@@H](CCCNC(N)=O)NC(=O)[C@H...
4,DB00080,Daptomycin,CCCCCCCCCC(=O)N[C@@H](CC1=CNC2=C1C=CC=C2)C(=O)...
...,...,...,...
2381,DB14104,NaN,CCCCC\C=C/C\C=C/CCCCCCCC(O)=O
2382,DB14116,NaN,O.[O--].[O--].[Mg++].[Mg++].[Al+3].[Al+3].[O-]...
2383,DB14151,NaN,[P]
2384,DB14156,NaN,CC1(C)C2CCC1(C)C(=O)C2


In [51]:
input_json = {
      "title": "Tylenol Extra Strength Caplets with 500 mg Acetaminophen, 100 ct",
      "description": "100-ct. of Tylenol Extra Strength Caplets with acetaminophen as pain reliever and fever reducerProvides temporary relief of minor aches and pains and helps reduce feverEach extra strength caplet contains 500 mg of acetaminophen, an effective pain relief ingredientRelieves common pain symptoms including headache, backache and menstrual crampsAlso works as a fever reducer and helps alleviate pain symptoms related to the common coldWhen used as directed, this pain reliever has an excellent safety profileThis oral pain reliever can be used by adults and children who are 12 years and olderFrom the #1 doctor-recommended brand of pain reliever, experience pain relief with TYLENOL"
}

In [70]:
# compounds[compounds == 'acetaminophen']
re.search('acetaminophen', input_json['description'].lower())

In [109]:
import re

drug_info_comb_fp = './data/Drug_info_combined.csv'
drug_info_comb = pd.read_csv(drug_info_comb_fp, index_col = 'Unnamed: 0')
compounds = drug_info_comb.Name.dropna()

def fuzzy_search(input_json, output_fp = './DDI_input.txt', pools = compounds):    
    description = input_json['description'].lower()
    out = []
    for compound in pools:
        pattern = compound.strip().lower()
        if re.search(pattern, description):
            out.append(compound)
    if output_fp:
        with open(output_fp, 'a') as fw:
            to_write = '\n'+'\n'.join(out)
            fw.write(to_write)

def ingest_input(input_json):
    fuzzy_search(input_json['current_drug'])
    for other_drug in input_json['other_drug']:
        fuzzy_search(other_drug['current_drug'])